In [1]:
import pandas as pd
import main

In [2]:
real_top = pd.read_csv('real_data_test\\top_df.csv', delimiter=';')
real_top = real_top.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_top['X']
del real_top['Y']
real_top.head(n=2)

,WELL_NAME_UWI,DEPTH_TOP
0,100,2548.81
1,101,2556.23


In [3]:
real_bottom = pd.read_csv('real_data_test\\bottom_df.csv', delimiter=';')
real_bottom = real_bottom.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_bottom['X']
del real_bottom['Y']
real_bottom.head(n=2)

,WELL_NAME_UWI,DEPTH_BOTTOM
0,100,2564.81
1,101,2571.71


In [4]:
# данные от организаторов
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'WC'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
del water_content_data['date_start']
water_content_data = water_content_data.rename(columns={'well_name': 'WELL_NAME'})
water_content_data.head(n=5)
water_content_data['WELL_NAME'] = water_content_data['WELL_NAME'].astype('str')
water_content_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 2 columns):
WELL_NAME    181 non-null object
WC           181 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.9+ KB


In [5]:
print(len(water_content_data['WELL_NAME'].value_counts()))

180


In [7]:
params_name = ['KINT', 'R05', 'R20', 'R14', 'R10', 'F07', 'F10', 'F14', 'R07', 'F20', 'F05', 'PHIT', 'MRES', 'SG', 'KGL', 'SXWB', 'GZ3', 'NPHI', 'GZ2', 'GZ4', 'GZ1', 'CILD', 'PROX', 'LLD', 'GZ7', 'GR:2', 'GR:1', 'APS', 'KPS', 'GZ5', 'SP:1', 'SP:2', 'W:1', 'W:2', 'NKTM:2', 'NKTM:1', 'CFTC:1', 'CFTC:2', 'MLM:2', 'MLM:1', 'RT:1', 'RT:2', 'GR:3']

In [12]:
count_val = 100
min_count_val_in_data = 30
for param in params_name:
    print('Convert ', param, 'start')
    param_name_file = (param.replace(':','')).lower()
    # Загружаем данные по параметру
    las_gis_file_name = 'AllGisParams\\fake_gis_las_' + param_name_file +  '_out.csv'
    las_data = pd.read_csv(las_gis_file_name, delimiter=';', low_memory=False)
    las_data['WELL_NAME'] = las_data['WELL_NAME'].astype('str')
    las_data['WELL_NAME_UWI'] = las_data['WELL_NAME_UWI'].astype('str')
    las_data.head(n=5)
    
    # Объединяем данные
    real_data = pd.merge(water_content_data, las_data, on='WELL_NAME')
    real_data = pd.merge(real_data, real_top, on='WELL_NAME_UWI')
    real_data = pd.merge(real_data, real_bottom, on='WELL_NAME_UWI')
    real_data_na = real_data.dropna()
    print(len(real_data['WELL_NAME'].value_counts()))
    
    # Преобразуем в данные для обучения
    dev_path = "..\\tasks\\task 6\\data\\dev\\"
    well_name_list = real_data_na['WELL_NAME_UWI'].value_counts().index.tolist()
    [x_values, y_values, y_names] = main.ConvertDataToLearning(real_data_na, dev_path, 10, 100)
    
    # Создаем DataFrame
    x_names = ['idx' + str(i) for i in range(len(x_values[0]))]
    data_x_values = pd.DataFrame(x_values, columns = x_names)
    data_y_values = pd.DataFrame(y_values, columns=['wc'])
    data_names = pd.DataFrame(y_names, columns=['WELL_NAME'])
    len(data_x_values)
    data_values = pd.merge(data_y_values, data_names, left_index=True, right_index=True)
    data_values = pd.merge(data_values, data_x_values, left_index=True, right_index=True)
    
    print('Convert', param, 'end')
    break
print(len(x_values))

Convert  KINT start
162
0  of  161
20  of  161
40  of  161
60  of  161
80  of  161
100  of  161
120  of  161
140  of  161
Convert KINT end
158


In [10]:
data_values.head()

,wc,WELL_NAME,idx0,idx1,idx2,idx3,idx4,idx5,idx6,idx7,...,idx90,idx91,idx92,idx93,idx94,idx95,idx96,idx97,idx98,idx99
0,0.08,554,121.0,130.440,141.60,149.08,150.00,150.000,150.16,152.520,...,171.40,172.760,175.12,176.480,177.00,175.600,170.88,165.160,158.44,151.440
1,0.01,542,106.0,102.125,97.15,90.75,82.80,73.500,65.50,60.300,...,110.75,111.525,112.60,114.075,114.85,116.250,117.00,117.000,117.00,117.000
2,0.33,563,141.5,149.725,153.70,158.10,162.60,164.625,161.80,156.375,...,103.75,117.950,134.40,147.750,154.75,158.125,158.40,154.625,149.40,140.575
3,0.51,319,107.0,112.005,114.51,117.02,120.36,123.025,124.51,125.000,...,113.70,112.515,112.00,112.310,113.98,118.950,121.98,123.990,125.33,126.165
4,0.10,555,156.9,172.300,186.25,195.51,200.56,203.000,203.00,201.990,...,171.10,172.775,174.88,175.970,176.00,176.000,176.78,175.485,172.30,168.700
